# 利用余弦相似度做日志文本分类

## 基本原理

数学中常用余弦定理来计算两条边的夹角，NLP(Natural Language Process)中可以用余弦定理来计算文本相似度。
利用余弦定理计算两个向量的夹角，求得的夹角 θ 越小（余弦值越大），说明两个向量越接近，计算公式如下:

**余弦相似度公式**

$$ 给定向量(\vec{a},\vec{b})，其余弦相似度为：$$
$$ cos(\vec{a},\vec{b})=\frac{\vec{a}.\vec{b}}{|\vec{a}|*|\vec{b}|} $$

其中的向量，在泛化到多维空间后为：
$$ \vec{a} = (x_1,x_2,...,x_n) $$ 
$$ \vec{b} = (y_1,y_2,...,y_n) $$ 

由此公式可以改写为：
$$ cos(\vec{a},\vec{b})=\frac{\sum_{i=1}^n(x_i y_i)} {\sqrt{\sum_{i=1}^n x_i^2}\sqrt{\sum_{i=1}^n y_i^2}} $$ 

**利用numpy包来实现余弦相似度公式**

In [1]:
import numpy as np

def get_cossimi(myx,myy):
    cos1=np.sum(myx*myy) #cos(a,b)=a*b/(|a||b|)
    cos21=np.sqrt(sum(myx*myx))
    cos22=np.sqrt(sum(myy*myy))
    return cos1/(float(cos21*cos22))


## 文本向量化

要用余弦相似度算法来比较文本的相似度，首先需要对文本进行向量化。最简单的方法就是统计文本中的词的发生次数，每个词对应一维，而该词的出现的频次则是这一维上的坐标。所有维度上的坐标则构成了在这个多维超空间下的向量。


## 统计WebSphere日志的词频

利用Pathon的collections.Counter来进行词频统计会非常有效率，但是对于WAS的日志输出，在每次输出的时候，都会加上前导字串：

[Datetime] [thread ID] [ClassName] [OutputFlag] <Output text]

例如：
[8/7/17 16:50:44:314 CST] 000006e1 yInfo.eservice.endpoint.PolicyInformationServicePortTypeImpl    I Executing operation retrievePolicy

特别注意的是：我们真正需要统计的是output text，前面的这些数据会形成干扰（出现频度太高了），所以，我们需要用正则表达式来对字串进行清理。

In [2]:
import re,collections

def get_nums(file):
    with open (file) as f:
        words_box=[]
        for line in f:                         
            strLong=cleanLine(line) #clean each line
            words_box.extend(strLong.strip().split())
    return collections.Counter(words_box)

# below method is used to clean the prefix string of each line which include:
#   datetime, thread id, output class name, output type(I/O/W/A)
def cleanLine(strLog):
    # match typical WAS output like [8/7/17 16:50:44:314 CST] 000006e1
    result1=re.match(r'\[[0-9:/ CST]*\]\s.{8}[0-9a-f]*',strLog)
    if result1:
        strLog=strLog[result1.end()+1:]  #如果匹配就去掉这部分
    else:
        return strLog
    
    # match typical WAS output 'class name I/O/W/A' part following above part
    result2=re.match(r'[.a-zA-Z0-9 ]*\s[IOWA]\s',strLog)

    if result2:
        strLog=strLog[result2.end():]
    return strLog



上面的程序已经实现了词频的统计，我们分拆同一个日志文件为三个：分别是1001-3000行，3001-5000行，5001-7000行，得到3个词频统计对象。这三个日志文件中，5001-7000行的日志中，包含了很多异常日志输出。

In [3]:
stats0=get_nums('messages_err.log')
print("messages_err:", stats0.most_common(20))

stats1=get_nums('msg1001-3000.log')
print("msg1001-3000:", stats1.most_common(20))

stats2=get_nums('msg3001-5000.log')
print("msg1001-3000:", stats2.most_common(20))
print("msg1001-3000:", (stats1+stats2).most_common(20))

stats3=get_nums('msg5001-7000.log').most_common(20)
print("msg1001-3000:", stats3)


messages_err: [('retrievePendings4E', 2282), ('at', 1444), ('Start', 1148), ('End', 1148), ('Pending', 1148), ('PendingV1.3', 1141), ('0,', 1137), ('Executing', 850), ('operation', 850), ('in', 657), ('retrievePolicy', 640), ('The', 472), ('class', 458), ('not', 454), ('be', 431), ('Cost:3', 404), ('or', 402), ('path', 398), ('policyContactsBeanList:', 397), ('found', 385)]
msg1001-3000: [('retrievePendings4E', 785), ('End', 394), ('Pending', 394), ('Start', 393), ('0,', 392), ('PendingV1.3', 392), ('Executing', 347), ('operation', 347), ('retrievePolicy', 256), ('policyContactsBeanList:', 164), ('Cost:3', 155), ('getPosInfo', 113), ('Time', 105), ('consumed', 105), ('in', 105), ('retrieveClaim', 83), ('GetClaimInfoService:', 83), ('Cost:2', 82), ('0', 61), ('1', 34)]
msg1001-3000: [('retrievePendings4E', 982), ('Start', 496), ('End', 496), ('Pending', 496), ('PendingV1.3', 491), ('0,', 489), ('Executing', 292), ('operation', 292), ('retrievePolicy', 242), ('Cost:2', 165), ('Cost:3', 1

In [4]:
# 定义一个方法，取得统计出来的词频中，给定关键词的发生数向量
def getWordFrequencyVector(statCounter,commonKeywords):
    vec=np.zeros(len(commonKeywords))
    dic=dict(statCounter)
    index=0
    for k in commonKeywords:
        vec[index]=dic.get(k,0) #取得关键词的发生数，如果无此关键词，则返回默认值：0
        index=index+1
    return vec

__Dimention__=40 # 这个变量设置词频向量的维度
commonKeywords=[] # 用于保存基准报文中的最频繁的(__Dimention__)个词及其向量
vec1=np.zeros(__Dimention__)
index=0
for k1,v1 in stats1.most_common(__Dimention__):
    commonKeywords.append(k1)
    vec1[index]=v1
    index=index+1

vec2=getWordFrequencyVector(stats2,commonKeywords)
vec3=getWordFrequencyVector(stats3,commonKeywords)

similarity1=float(get_cossimi(vec1,vec2))*100
similarity2=float(get_cossimi(vec1,vec3))*100
similarity3=float(get_cossimi(vec2,vec3))*100
print("The degree of similarity of logs(1 vs. 2) is: %6.2f%s" % (similarity1,'%'))
print("The degree of similarity of logs(1 vs. 3) is: %6.2f%s" % (similarity2,'%'))
print("The degree of similarity of logs(2 vs. 3) is: %6.2f%s" % (similarity3,'%'))


The degree of similarity of logs(1 vs. 2) is:  98.86%
The degree of similarity of logs(1 vs. 3) is:  73.49%
The degree of similarity of logs(2 vs. 3) is:  77.33%


可以注意到：
* 前2000行与中间2000行非常相似，相似度达到：98.88%，
* 第一、二段和第三段的相似度下降到74%和77%
* 增加维度可以提高精度，但超过20后，相似度基本无变化

## 利用这个我们能做什么？

对于日志文件，我们面临的问题是：数量巨大且分散导致检索困难。在有了ELK这样的工具后，此问题可以得到很大的改善。然而，及时、即时发现日志中的异常情况，仍然是一个很大的难题。虽然可以用关键字过滤和搜索的方式，在ELK里面预警，但这是需要大量工作、对应用非常了解并具有足够的预见性。

利用上述的文本比较，可以提供一个新的日志处理思路，总的步骤是：
* 利用若干段正常情况下的日志文本作为样本，得到对应的标准样本集和词频向量
* 每隔一段时间，获取最新的日志数据（比如尾部的1000行），统计分词的数量
* 根据不同标准样本定义的多维空间，获得日志数据的词频向量
* 获得当前样本与各标准样本之间的相似度，如果和任何一个都不相似（低于预设阈值），则告警
